In [0]:
pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 40kB/s 
     |████████████████████████████████| 3.8MB 30.9MB/s 
     |████████████████████████████████| 450kB 37.6MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.8.2 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exist

# Stage 1: Importing project dependencies

In [0]:
import math
import random
import numpy as np
import statistics
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tqdm
from collections import deque
import os.path


# Stage 2: Building the AI Trader network

In [0]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=5, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=5550)
    self.model_name = model_name
    self.inventory_gp = []
    self.inventory_sqph = []
    self.gamma = 0.95
    self.epsilon = 1
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.LR = 0.0001
    self.TAU = 1e-3       # for soft update of target parameters
    
    self.local_network = self.model_builder()
    self.target_network = self.model_builder()
    
  def model_builder(self):
    
      model = tf.keras.models.Sequential()
      
      model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
      
      model.add(tf.keras.layers.Dense(units=64, activation='relu'))
      
      model.add(tf.keras.layers.Dense(units=128, activation='relu'))
      
      model.add(tf.keras.layers.Dense(units=256, activation='relu'))

      model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
      
      model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=self.LR))
      
      return model
    
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.local_network.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    #for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      #batch.append(self.memory[i])

    batch = random.sample(self.memory, 32)  
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.target_network.predict(next_state)[0])
        
      target = self.local_network.predict(state)
      target[0][action] = reward
      
      self.local_network.fit(state, target, epochs=1, verbose=0)
      
    #if self.epsilon > self.epsilon_final:
      #self.epsilon *= self.epsilon_decay

    self.epsilon = max(self.epsilon_final, self.epsilon_decay*self.epsilon)

    self.soft_update()

  def soft_update(self):
    for target_params, local_params in zip(self.target_network.trainable_variables, self.local_network.trainable_variables):
      target_params.assign(target_params * (1 - self.TAU) + local_params * self.TAU)


# Stage 3: Dataset preprocessing

##helper function

In [0]:
def sigmoid(x):
  return (1 / (1 + np.exp(-x)))

##Price format function

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

##Loading LSTM for prediction and preprocessing it

In [5]:
#Load the pretrained model
with open('SQPH prediction.json', 'r') as f:
    modelsqph_json = f.read()

model_sqph = tf.keras.models.model_from_json(modelsqph_json)

with open('GP prediction.json', 'r') as f:
    modelgp_json = f.read()

model_gp = tf.keras.models.model_from_json(modelgp_json)

# load weights into new model
model_sqph.load_weights("SQPH prediction.json.h5")
model_gp.load_weights("GP prediction.json.h5")

#preprocessing
dataset_test_sqph = pd.read_csv('SQPH Historical Data2017-2018.csv')
real_stock_price_sqph = dataset_test_sqph.iloc[:, 1:2].values
dataset_train_sqph = pd.read_csv('SQPH Historical Data 2007-2017.csv')
dataset_total_sqph = pd.concat((dataset_train_sqph['Price'], dataset_test_sqph['Price']), axis = 0)

dataset_test_gp = pd.read_csv('GRAE Historical Data 2017-2018.csv')
real_stock_price_gp = dataset_test_gp.iloc[:, 1:2].values
dataset_train_gp = pd.read_csv('GRAE Historical Data 2007-2017.csv')
dataset_total_gp = pd.concat((dataset_train_gp['Price'], dataset_test_gp['Price']), axis = 0)

inputs_sqph = dataset_total_sqph[len(dataset_total_sqph) - len(dataset_test_sqph) - 60:].values
inputs_gp = dataset_total_gp[len(dataset_total_gp) - len(dataset_test_gp) - 60:].values
inputs_sqph = inputs_sqph.reshape(-1,1)
inputs_gp = inputs_gp.reshape(-1,1)
sc = MinMaxScaler(feature_range = (0, 1))
inputs_sqph = sc.fit_transform(inputs_sqph)
inputs_gp = sc.transform(inputs_gp)
X_test_sqph = []
for i in range(60, 60+dataset_test_sqph.shape[0]):
    X_test_sqph.append(inputs_sqph[i-60:i, 0])
X_test_sqph = np.array(X_test_sqph)
X_test_sqph = np.reshape(X_test_sqph, (X_test_sqph.shape[0], X_test_sqph.shape[1], 1))

X_test_gp = []
for i in range(60, 60+dataset_test_gp.shape[0]):
    X_test_gp.append(inputs_gp[i-60:i, 0])
X_test_gp = np.array(X_test_gp)
X_test_gp = np.reshape(X_test_gp, (X_test_gp.shape[0], X_test_gp.shape[1], 1))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


##State creator

In [0]:
def state_creator(data_gp, data_sqph, timestep, window_size,obj):
  
  starting_id = timestep - window_size + 1
  predict_sqph = sc.inverse_transform(model_sqph.predict(np.reshape(X_test_sqph[timestep],(X_test_sqph[timestep].shape[1],X_test_sqph[timestep].shape[0],1))))
  predict_gp = sc.inverse_transform(model_gp.predict(np.reshape(X_test_gp[timestep],(X_test_gp[timestep].shape[1],X_test_gp[timestep].shape[0],1))))
  if starting_id >= 0:
    windowed_data_gp = data_gp[starting_id:timestep+1]
    windowed_data_sqph = data_sqph[starting_id:timestep+1]
  else:
    windowed_data_gp = - starting_id * [data_gp[0]] + list(data_gp[0:timestep+1])
    windowed_data_sqph = - starting_id * [data_sqph[0]] + list(data_sqph[0:timestep+1])

  state = []
  for i in range(window_size - 1):
    state.append(windowed_data_gp[i+1] - windowed_data_gp[i])
    state.append(windowed_data_sqph[i+1] - windowed_data_sqph[i])
    
  state.append(predict_sqph[0,0])
  state.append(predict_gp[0,0])
  state.append(len(obj.inventory_gp))
  state.append(len(obj.inventory_sqph))
  return sigmoid(np.array([state]))


##Loading a dataset

In [0]:
data_sqph = list(pd.read_csv( "SQPH Historical Data2017-2018.csv")["Price"])
data_gp = list(pd.read_csv( "GRAE Historical Data 2017-2018.csv")["Price"])

# Stage 4: Training the AI Trader

##Setting hyper parameters

In [0]:
window_size = 10
episodes = 1000
onestep_gp_stock = 1
onestep_sqph_stock = 1
gp_stock_count = 1
sqph_stock_count = 1
sharpe_ratio_list = []
portfolio_list = []

batch_size = 32
data_samples = len(data_gp) - 1


##Defining the Trader model

In [0]:
trader = AI_Trader(2*window_size+onestep_gp_stock+onestep_sqph_stock+gp_stock_count+sqph_stock_count)

In [0]:
trader.local_network.load_weights("AI_trader_local_network_weights.json.h5")
trader.target_network.load_weights("AI_trader_target_network_weights.json.h5")

##Training loop

In [10]:
for episode in range(1, episodes + 1):  
  print("Episode: {}/{}".format(episode, episodes))
  state = state_creator(data_gp, data_sqph, 0, window_size + 1,trader)
  total_profit = 0
  trader.inventory_gp = []
  trader.inventory_sqph = []
  stock_return_list = []
      
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data_gp, data_sqph, t+1, window_size + 1,trader)
    reward = 0
    if action == 1: #Buying gp
        trader.inventory_gp.append(data_gp[t])
        print("AI Trader bought: ", stocks_price_format(data_gp[t]))

    if action == 2 and len(trader.inventory_gp) > 0: #Selling gp
      buy_prices = trader.inventory_gp.pop(0)
      total_profit += (data_gp[t] - buy_prices)
      reward = max((data_gp[t] - buy_prices),-1)
      stock_return = (data_gp[t] - data_gp[t-1])/data_gp[t-1]
      stock_return_list.append(stock_return)
      print("AI Trader sold: ", stocks_price_format(data_gp[t]), " Profit: " + stocks_price_format(data_gp[t] - buy_prices))

    if action == 3: #Buying sqph
        trader.inventory_sqph.append(data_sqph[t])
        print("AI Trader bought: ", stocks_price_format(data_sqph[t]))

    if action == 4 and len(trader.inventory_sqph) > 0: #Selling sqph
      buy_prices = trader.inventory_sqph.pop(0)
      total_profit += (data_sqph[t] - buy_prices)
      reward = max((data_sqph[t] - buy_prices),-1)
      stock_return = (data_sqph[t]-data_sqph[t-1])/data_sqph[t-1]
      stock_return_list.append(stock_return)
      print("AI Trader sold: ", stocks_price_format(data_sqph[t]), " Profit: " + stocks_price_format(data_sqph[t] - buy_prices))   
    
    if len(portfolio_list) and portfolio_list[0]:
        portfolio = ((sum(trader.inventory_gp) + sum(trader.inventory_sqph))-portfolio_list[0])/portfolio_list[0]
        portfolio_list.append(portfolio)
        
    else:
        portfolio = sum(trader.inventory_gp) + sum(trader.inventory_sqph)
        portfolio_list.append(portfolio)
        
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
      sharpe_ratio = statistics.mean(portfolio_list)/statistics.stdev(portfolio_list)
      sharpe_ratio_list.append(sharpe_ratio)
    
    f = open("file2.txt", "w")
    f.write(str(trader.epsilon))
    f.close()
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
    
    #if episode%100==0:      
    ### Saving the architecture (topology) of the network
    trader_local = trader.local_network.to_json()
    with open("AI_trader_local_network.json", "w") as json_file:
        json_file.write(trader_local)

    trader_target = trader.target_network.to_json()
    with open("AI_trader_target_network.json", "w") as json_file:
        json_file.write(trader_target)

    ### Saving network weights
    trader.local_network.save_weights("AI_trader_local_network_weights.json.h5")
    trader.target_network.save_weights("AI_trader_target_network_weights.json.h5")

Episode: 1/1000


  2%|▏         | 4/241 [00:00<01:01,  3.84it/s]

AI Trader bought:  $ 361.000000
AI Trader sold:  $ 360.400000  Profit: - $ 0.600000
AI Trader bought:  $ 248.700000


  5%|▍         | 11/241 [00:00<00:28,  8.12it/s]

AI Trader sold:  $ 250.800000  Profit: $ 2.100000
AI Trader bought:  $ 247.900000


...

AI Trader bought:  $ 429.400000


 60%|██████    | 145/241 [10:51<07:29,  4.68s/it]

AI Trader sold:  $ 429.700000  Profit: $ 0.300000


 61%|██████▏   | 148/241 [11:05<07:12,  4.65s/it]

KeyboardInterrupt: ignored

##sharpe ratio plot

In [0]:
plt.plot(sharpe_ratio_list)
plt.title('sharpe_ratio (Training set)')
plt.xlabel('time')
plt.ylabel('sharpe ratio')
plt.show()

# Rough section

In [0]:







"""**loading the weight**"""

if os.path.exists("AI_trader_weights_v3.0_1100.json.h5")== True:
    trader.model.load_weights("AI_trader_weights_v3.0_1100.json.h5")

"""**loading the test set**"""

data_sqph_test = list(pd.read_csv( "SQPH Historical Data 2018-2019.csv")["Price"])
data_gp_test = list(pd.read_csv( "GRAE Historical Data 2018-2019.csv")["Price"])

"""**Loading the pretrained LSTM for prediction**"""

with open('SQPH prediction.json', 'r') as f:
    modelsqph_json = f.read()

model_sqph = tf.keras.models.model_from_json(modelsqph_json)

with open('GP prediction.json', 'r') as f:
    modelgp_json = f.read()

model_gp = tf.keras.models.model_from_json(modelgp_json)

# load weights into new model
model_sqph.load_weights("SQPH prediction.json.h5")
model_gp.load_weights("GP prediction.json.h5")

dataset_test_sqph = pd.read_csv('SQPH Historical Data 2018-2019.csv')
real_stock_price_sqph = dataset_test_sqph.iloc[:, 1:2].values
dataset_train_sqph = pd.read_csv('SQPH Historical Data2017-2018.csv')
dataset_total_sqph = pd.concat((dataset_train_sqph['Price'], dataset_test_sqph['Price']), axis = 0)

dataset_test_gp = pd.read_csv('GRAE Historical Data 2018-2019.csv')
real_stock_price_gp = dataset_test_gp.iloc[:, 1:2].values
dataset_train_gp = pd.read_csv('GRAE Historical Data 2017-2018.csv')
dataset_total_gp = pd.concat((dataset_train_gp['Price'], dataset_test_gp['Price']), axis = 0)

inputs_sqph = dataset_total_sqph[len(dataset_total_sqph) - len(dataset_test_sqph) - 60:].values
inputs_gp = dataset_total_gp[len(dataset_total_gp) - len(dataset_test_gp) - 60:].values
inputs_sqph = inputs_sqph.reshape(-1,1)
inputs_gp = inputs_gp.reshape(-1,1)
sc = MinMaxScaler(feature_range = (0, 1))
inputs_sqph = sc.fit_transform(inputs_sqph)
inputs_gp = sc.transform(inputs_gp)
X_test_sqph = []
for i in range(60, 60+dataset_test_sqph.shape[0]):
    X_test_sqph.append(inputs_sqph[i-60:i, 0])
X_test_sqph = np.array(X_test_sqph)
X_test_sqph = np.reshape(X_test_sqph, (X_test_sqph.shape[0], X_test_sqph.shape[1], 1))

X_test_gp = []
for i in range(60, 60+dataset_test_gp.shape[0]):
    X_test_gp.append(inputs_gp[i-60:i, 0])
X_test_gp = np.array(X_test_gp)
X_test_gp = np.reshape(X_test_gp, (X_test_gp.shape[0], X_test_gp.shape[1], 1))

"""**Testing the model**"""

for episode in range(1, 2):
  money = 100000
    
  state = state_creator(data_gp_test, data_sqph_test, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory_gp = []
  trader.inventory_sqph = []

  for t in tqdm(range(len(data_sqph_test)-1)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data_gp_test, data_sqph_test, t+1, window_size + 1)
    
    if action == 1: #Buying gp
        if money >= data_gp_test[t]:
            trader.inventory_gp.append(data_gp_test[t])
            money -= data_gp_test[t]
            print("AI Trader bought: ", stocks_price_format(data_gp_test[t]),"Total money: ",money)

    elif action == 2 and len(trader.inventory_gp) > 0: #Selling gp
      buy_price = trader.inventory_gp.pop(0)
      
      total_profit += data_gp_test[t] - buy_price
      money += total_profit
      print("AI Trader sold: ", stocks_price_format(data_gp_test[t]), " Profit: " + stocks_price_format(data_gp_test[t] - buy_price),"Total money: ",money)

    if action == 3: #Buying sqph
        if money >= data_sqph_test[t]:
            trader.inventory_sqph.append(data_sqph_test[t])
            money -= data_sqph_test[t]
            print("AI Trader bought: ", stocks_price_format(data_sqph_test[t]),"Total money: ",money)

    elif action == 4 and len(trader.inventory_sqph) > 0: #Selling sqph
      buy_price = trader.inventory_sqph.pop(0)
      
      total_profit += data_sqph_test[t] - buy_price
      money += total_profit
      print("AI Trader sold: ", stocks_price_format(data_sqph_test[t]), " Profit: " + stocks_price_format(data_sqph_test[t] - buy_price),"Total money: ",money)
    
    if t == len(data_sqph_test)-2 :
      done = True
    else:
      done = False
          
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("TOTAL money: {}".format(money))
      print("########################")

